In [1]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

import sys
sys.path.append('/content/gdrive/My Drive/workspace')

%cd 'gdrive/My Drive/workspace'

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive
/content/gdrive/My Drive/workspace


In [0]:
import LBP
import preprocessing as prep

import numpy as np
from collections import Counter
import matplotlib.pyplot as plt


import dlib

from PIL import Image


from importlib import reload

_ = reload(prep)

from sklearn.model_selection import train_test_split


import helpers
from os.path import join
reload(helpers)
import math

In [0]:
predictor_path = 'shape_predictor_68_face_landmarks.dat'
detector_path = 'mmod_human_face_detector.dat'
root_path = 'lfw'
people_txt = 'people.txt'

In [0]:
def ranges_of(arr):
    for i  in range(len(arr)-1):
        yield arr[i], arr[i+1]

def as_if_a_vector(counter):
    for i in range(256):
        yield counter[i]

def histograms(arr, dimx=8, dimy=8):
    x, y = arr.shape
    for xlow, xhigh in ranges_of(np.linspace(0, x, num=dimx+1, dtype=np.int)):
        for ylow, yhigh in ranges_of(np.linspace(0, y, num=dimy+1, dtype=np.int)):
            yield Counter(arr[xlow:xhigh, ylow:yhigh].flatten())
            
def dist_of_counters(c1, c2):
    c = c1 if len(c1)>len(c2) else c2
    return math.sqrt(sum((c1[k] - c2[k])**2 for k in c.keys()))

def eu_dist(hists1, hists2):
    dists = [dist_of_counters(c1,c2) for c1, c2 in zip(hists1, hists2)]
    return math.sqrt(sum(d**2 for d in dists))

def lbp_hists(face_file_path):
    img = dlib.load_rgb_image(face_file_path)
    funneled_image_arr = prep.cropped_and_aligned(img, cnn_face_detector, sp,method='cnn')
    fun_image = Image.fromarray(funneled_image_arr)
    lbp_img = LBP.to_LBP(fun_image)
    lbp_arr = np.asarray(lbp_img,dtype=np.uint8)
    return list(histograms(lbp_arr))
        


In [0]:
cnn_face_detector = dlib.cnn_face_detection_model_v1(detector_path)

sp = dlib.shape_predictor(predictor_path)

data load start here

In [0]:
df = helpers.load_names(people_txt, 6,20)
names = set(df['name'])
df['image_path'] = df.apply(lambda row: join(root_path, row['name'], row['name']+'_'+str(row['image_num']).zfill(4) + '.jpg'),axis=1)

In [7]:
df.shape, len(names)

((2502, 3), 254)

In [0]:

rest, test = train_test_split(df, test_size=0.2, stratify=df['name'])
train, _  = train_test_split(rest, test_size=0.2, stratify=rest['name'])

In [9]:
print(test.shape, train.shape), len(set(train.name)), len(set(test.name))

(501, 3) (1600, 3)


(None, 254, 254)

In [11]:
train['lbph_features'] = train['image_path'].apply(lbp_hists)
test['lbph_features'] = test['image_path'].apply(lbp_hists)

/content/gdrive/My Drive/workspace/preprocessing.py:99: UserWarning: Found 2 faces, using the one with the highest confidence score.
  warn('Found {} faces, using the one with the highest confidence score.'.format(len(dets)))
/content/gdrive/My Drive/workspace/preprocessing.py:99: UserWarning: Found 3 faces, using the one with the highest confidence score.
  warn('Found {} faces, using the one with the highest confidence score.'.format(len(dets)))
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try usi

In [12]:

p = 0
n = 0
for  i, name1, _, path1, features1 in test.itertuples():
    closest = ''
    mindist = float('Inf')
    for _, name2, _, path2, features2 in train.itertuples():
        dist = eu_dist(features1, features2)
        if dist < mindist:
            mindist = dist
            closest = name2
    print(name1==closest, name1, closest)
    if closest == name1:
        p+=1
    else:
        n+=1
        
    
    

False John_Paul_II Venus_Williams
False Charles_Moose Pervez_Musharraf
False Hugh_Grant Shimon_Peres
False Jose_Serra Robert_Duvall
True Hu_Jintao Hu_Jintao
False Mohammad_Khatami Vojislav_Kostunica
False Richard_Armitage Hu_Jintao
False Paradorn_Srichaphan Ray_Romano
False Halle_Berry Salma_Hayek
False Ian_Thorpe Albert_Costa
True Robert_De_Niro Robert_De_Niro
False Zinedine_Zidane Pierce_Brosnan
False Tom_Cruise Jean-David_Levitte
False Bob_Hope Albert_Costa
False Adrien_Brody Bill_Simon
False Dick_Cheney Ron_Dittemore
True Yasser_Arafat Yasser_Arafat
True Kamal_Kharrazi Kamal_Kharrazi
True Oscar_De_La_Hoya Oscar_De_La_Hoya
False Tang_Jiaxuan Geoff_Hoon
True George_HW_Bush George_HW_Bush
False Shimon_Peres John_Abizaid
False Jean_Charest Ron_Dittemore
True Meryl_Streep Meryl_Streep
True Roger_Federer Roger_Federer
False Anna_Kournikova Juan_Pablo_Montoya
True Abdullah_Gul Abdullah_Gul
False Dennis_Kucinich Pervez_Musharraf
False Luis_Horna Ron_Dittemore
False Heizo_Takenaka Bill_Simo

In [13]:
p,n, p/(p+n)

(110, 391, 0.21956087824351297)